In [1]:
from fasthtml.common import *
from monsterui.all import *

import os, pty, tty, fcntl, termios, select, threading, asyncio

In [2]:
from fasthtml.jupyter import *

In [3]:
hdrs = (
        Theme.blue.headers(),                 # MonsterUI theme
        Script(src="/static/node_modules/@xterm/xterm/lib/xterm.js", type="module"),    # xterm glue code
        Script(src="/static/terminal.js", type="module"),    # xterm glue code
        Link(rel="stylesheet", href="/static/node_modules/@xterm/xterm/css/xterm.css")
)
app, rt = fast_app(hdrs=hdrs,routes=(Mount('/static', StaticFiles(directory=f'./static')),))

In [4]:
if 'server' in globals(): server.stop()
server = JupyUvi(app)
Show = partial(HTMX, app=app)

Sending: 
The default interactive shell is now zsh.
To update your account to use zsh, please run `chsh -s /bin/zsh`.
For more details, please visit https://support.apple.com/kb/HT208050.
(uvws) (uvws) ls
main.ipynb        package.json      test.txt
package-lock.json static
(uvws) (uvws) Received message: l
Received message: s
Received message: 
Sending: 
Sending: main.ipynb        package.json      test.txt
package-lock.json static
Sending: (uvws) (uvws) Received message: 
Sending: 
Sending: (uvws) (uvws) Received message: Terminal Demo
Sending: Terminal Demo
Sending: bash: Terminal: command not found
Sending: (uvws) (uvws) Received message: Terminal Demo
Sending: Terminal Demo
Sending: bash: Terminal: command not found
Sending: (uvws) (uvws) Received message: 
Sending: (uvws) (uvws) Sending: 
Received message: 
Sending: 
Sending: (uvws) (uvws) Received message: (uvws) (uvws) (uvws) (uvws) 
Sending: (uvws) (uvws) (uvws) (uvws) Received message: 
Sending: bash: syntax error near unexpe

Exception in thread Thread-6 (read_thread):
Traceback (most recent call last):
  File "/Users/nathan/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/nathan/aai-ws/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/nathan/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_39769/1160411317.py", line 4, in read_thread
  File "/Users/nathan/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/asyncio/base_events.py", line 686, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/var/folders/5f/gb9vtfjd68q8pwth3s3t67rr0000gn/T/ipykernel_39769/2349332962.py", line 10, in _read
  File "/Users/nathan/aa

Received message: l
Received message: s
Received message: 
Sending: 
Sending: main.ipynb        package.json      test.txt
package-lock.json static
Sending: (uvws) (uvws) Received message: l
Received message: s
Received message: 
Sending: 
Sending:Sending: package-lock.json static
 main.ipynb        package.json      test.txt
Sending: (uvws) (uvws) Received message: c
Received message: a
Received message: t
Received message:  
Received message: p
Received message: a
Received message: k
Received message: 
Sending:Received message: c
Received message: k
Received message: 
Sending:Received message: 
Sending:Received message: 
Sending:Received message: 
Sending:Received message: 
Sending:Received message: 
Sending:Received message: 
Sending:Received message: 
Sending: 
Sending: bash: c: command not found
Sending: (uvws) (uvws) Received message: c
Received message: a
Received message: t
Received message:  
Received message: p
Received message: a
Received message: c
Received message:

In [5]:
import os,pty,select
from threading import Thread

In [6]:
def spawn_shell():
    pid, master_fd = pty.fork()
    if pid == 0:               # child
        os.execvp(os.environ.get("SHELL", "/bin/bash"), ["bash"])
    # set master fd non‑blocking
    # flags = fcntl.fcntl(master_fd, fcntl.F_GETFL)
    # fcntl.fcntl(master_fd, fcntl.F_SETFL, flags | os.O_NONBLOCK)
    return master_fd

In [7]:
fd = spawn_shell()

In [8]:
async def _read(fd, send=None): 
    "Read from PTY and print output"
    while True:
        try:
            r,_,_ = select.select([fd], [], [], 0.1)
            if r:
                d = os.read(fd, 1024).decode()
                if len(d) > 1:
                    print('Sending:', d, end='', flush=True)
                    if send: await send(d)
                # await send(d)
        except OSError: break

In [9]:
def read_thread(fd, send=None):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(_read(fd, send))
    loop.close()

# Start the thread

In [10]:
os.write(fd, b'ls\n')

3

In [11]:
async def on_conn(send):
    thread = Thread(target=read_thread, args=(fd, send))
    thread.daemon = True  # Make the thread daemon so it exits when the main program exits
    thread.start()
    await send('Hello, user!')

In [12]:
async def on_disconn(*args): ...
    # if hasattr(on_message, 'reader_task'):
    #     on_message.reader_task.cancel()
    # if hasattr(on_message, 'fd'):
    #     os.close(on_message.fd)

In [13]:
@app.ws('/ws', conn=on_conn, disconn=on_disconn)
async def on_message(msg: str, send):
    # Write incoming message to existing PTY
    print(f"Received message: {msg}")
    os.write(fd, msg.encode())

    # Optionally, you might want to await a short sleep to allow read operations to occur
    await asyncio.sleep(0.1)

In [14]:
@rt
def index():
    card = Card(
        Div(id='terminal', cls="w-full h-[600px] bg-black text-white"),
        header=H4("Web Terminal"),
        cls="mt-6")
    return Titled("Terminal Demo", card)

In [15]:
Show(index(), link=True)

In [5]:
def qshow(*c, height='auto'):
    "Quick show"
    doc = Html(Head(*hdrs), Body(*c))
    show(doc, iframe=True, height=height)